# Data Cleaning

In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests

In [3]:
def get_showroom_data( url, headers):

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text, "html.parser")

    total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
    page_number = np.round( int( total_item ) / 36 )
    urlfull = url + '?page-size=' + str( int( page_number*36 ) )
    page = requests.get( urlfull , headers=headers )
    soup = BeautifulSoup( page.text, "html.parser")
    total_item = [p.get("data-articlecode") for p in soup.find_all("article", class_='hm-product-item')]

    products_ids = []

    for i in total_item:

        url = f'https://www2.hm.com/en_us/productpage.{i}.html'

        page = requests.get( url, headers=headers )

        soup = BeautifulSoup( page.text , 'html.parser' )

        aux = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')
        aux = [p.get('data-articlecode') for p in aux]

        products_ids.append(aux)


    products_ids2 = []

    for i in products_ids:
        products_ids2 += i

    products_ids2 = list(set(products_ids2))


    cols = ['product_id', 'product_name', 'composition', 'color', 'size', 'fit', 'additional material information']
    df_pattern = pd.DataFrame( columns = cols)

    for i in products_ids2:

        url = f'https://www2.hm.com/en_us/productpage.{i}.html'
        page = requests.get( url, headers=headers )
        soup = BeautifulSoup( page.text , 'html.parser' )

        product_list = soup.find_all('div', class_='details-attributes-list-item')

        # remove empty strings
        product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]

        dici = {}
        for l2 in product_composition:
            dici[l2[0].lower()] = l2[1]

        name_price = soup.find_all('section', class_='product-name-price')[0]
        product_name = name_price.find('h1').text
        product_price = name_price.find('span', class_='price-value').text

        dici['product_name'] = product_name
        dici['price'] = product_price

        aux = soup.find_all('a', class_='filter-option miniature active')[0]
        dici['color'] = aux.get('data-color')


        df_aux = pd.DataFrame.from_dict(dici, orient='index').T
        df_aux = df_aux.rename(columns={'art. no.':"product_id"})

        df_pattern = pd.concat( [df_pattern,df_aux] )

    df_scraped = df_pattern.copy()
    df_scraped['style_id'] = df_scraped['product_id'].str[:-3]
    df_scraped['color_id'] = df_scraped['product_id'].str[-3:]
    df_scraped = df_scraped.sort_values('product_id')
    df_scraped['scrapy_datetime']=datetime.today()

    df_scraped.to_csv("df_collected.csv", index=False)

    return df_scraped

'88'

In [ ]:
def get_product_details( df_scrapped ):

    df_clean = df_scrapped.copy()

    for i in ['product_name', 'fit', 'description', 'material']:
        df_clean[i] = df_clean[i].apply( lambda x: x.replace(" ", "_").lower())
        df_clean[i] = df_clean[i].apply( lambda x: x.replace("/", "_").lower())

    df_clean['composition'] = df_clean['composition'].str.lower()
    df_clean['price'] = df_clean['price'].apply( lambda x: x.replace("$", "")).astype(float)
    df_clean['scrapy_datetime'] = pd.to_datetime(df_clean['scrapy_datetime'], format = "%Y-%m-%d %H:%M:%S")

    df_clean = df_clean.drop(['size'], axis=1)

    for i in ['style_id', 'color_id', 'product_id']:
        df_clean[i] = df_clean[i].astype(int)

    df_clean = df_clean.drop_duplicates()
    df_clean = df_clean.reset_index(drop=True)

    return df_clean

In [15]:
query_drop = """
    DROP TABLE products
"""

conn = sqlite3.connect( "hm_db.sqlite" )
cursor = conn.execute( query_drop )
conn.commit()

query_showroom_schema = """

    CREATE TABLE products (
    product_id        INTEGER,
    product_name      TEXT,
    composition       TEXT,
    color             TEXT,
    size              TEXT,
    fit               TEXT,
    material          TEXT,
    description       TEXT,
    price             REAL,
    style_id          INTEGER,
    color_id          INTEGER,
    scrapy_datetime   INTEGER
)

"""

cursor = conn.execute( query_showroom_schema )
conn.commit()
conn.close()

In [ ]:
def insert_into_database(df_clean):
    conn = create_engine( 'sqlite:///hm_db.sqlite' , echo=False)
    df_clean.to_sql( 'products' , con=conn, if_exists='append', index=False)

In [ ]:
query = """
SELECT * FROM products
"""

df = pd.read_sql_query( query, conn)
df.head()

In [ ]:
if __name__ == "__main__":
    
    url = 'https://www2.hm.com/en_us/men/products/jeans.html'
    headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

    
    # Extraction
    
    data_scraped = get_showroom_data( url, headers )
    
    # Transformation
    df_clean = get_product_details( data_scraped )
    
    # Carga
    data = insert_into_database( df_clean )
        